In [1]:
import pandas as pd
import numpy as np
import collections

path = "/Users/yanbingliu/Dropbox/infocom23-dataset/interm_processed"
interm_path = "/Users/yanbingliu/Dropbox/infocom23-dataset/interm_result"

grid_size = 0.0005
sample_limit = 0
time_limit = 0

unit_lat = 0.0005
unit_lon = 0.0005
digits = 4

operator_list = ["A", "V", "T"]
#operator_list = ["A", "T"]
#operator_list = ["V"]
region_list=["A-R1", "A-R2", "A-R3", "A-R4", "A-R5", "V-R6", "V-R7", "V-R8", "V-R9", "V-R10", "V-R11", "T-R12", "T-R13"]
#region_list=["A-R1", "A-R2", "A-R3", "A-R4", "A-R5"]
#region_list=["A-R1"]
#region_list=["V-R6", "V-R7", "V-R8", "V-R9", "V-R10", "V-R11"]

#ue_type_list = ["Pixel5", "Pixel4a"]
ue_type_list = ["Pixel5"]
#ue_type_list = ["Pixel4a"]
#target_grid_type_list = ["all", "R1A", "R1B", "R1C"]
target_grid_type_list = ["all"]
rat_list = ["5gm", "5gs"]

dataset_stat = []

region_res = {}

rsrp_threshold = -110

thput_sample_thres = 10
dominate_thres = 0.01

thput_factor_list = [0.1,0.2,0.3,0.4,0.5, 0.6, 0.7, 0.8, 0.9]

omega_list = [0.1, 0.3, 0.5, 0.7, 0.9]
gamma_list = [10, 20, 30, 40, 50]

freq_to_bandwidth = {675:15, 850:20, 1025:5, 5110:10, 5330:10, 9820:10, 9840:5, 
52740:20, 52940:20, 52941:20, 53139:20, 53140:20, 53141:20, 53339:20, 53341:20, 53539:20,
66486:10, 66661:5, 66686:10, 66936:10, 67086:10}

distribution = {}

def get_grid(s, unit):
    return round(int(float(s) / unit) * unit, digits)

for operator in operator_list:
    for ue_type in ue_type_list:
        mccmnc = ""
        if operator.find("A") >= 0:
            mccmnc = "310410"
        elif operator.find("V") >= 0:
            mccmnc = "311480"
        elif operator.find("T") >= 0:
            mccmnc = "310260"

        bw_freq = {}
        lte_bw_freq = {}
            
        total_instance_num = 0
        
        nr_cell_gps = {}
        max_bw_grid = {}
        cell_bw_grid = {}
        cell_set_bw_grid = {}
        
        cell_bw = {}
        
        mw_rsrp_grid = {}
        rsrp_grid = {}
        
        rsrp_list_0_10 = []
        rsrp_list_10_20 = []
        rsrp_list_20_30 = []
        rsrp_list_30_40 = []
        rsrp_list_40_50 = []
        rsrp_list_50_60 = []
        rsrp_list_60_70 = []
        rsrp_list_70_80 = []
        rsrp_list_80_90 = []
        rsrp_list_90_100 = []
        
        rsrp_list_0_20 = []
        rsrp_list_20_40 = []
        rsrp_list_40_60 = []
        rsrp_list_60_80 = []
        rsrp_list_80_100 = []
        
        sub6_rsrp_grid = {}
        
        mw_thput_list = []
        sub6_thput_list = []
        lte_thput_list = []

        for region in region_list:
            if region.find(operator) < 0:
                continue
            
            print(region)
            
            distribution[region] = []
            
            with open(interm_path + "/" + region + "_gps_cellset_thput_0.1_taskround_list_" + mccmnc + "_all_20210405_20220127.csv.csv", 'r', encoding='utf-8-sig') as lines:
                for line in lines:
                    if line.find("lat") >= 0:
                        continue
                    items = line.strip().split(',')
                    cell_set_type = items[20]
                    
                    bw = int(items[-1])

                    if bw <= 0:
                        continue
                        
                    traffic_mode = items[-2]
                    confidence_level = items[-3]
                    
                    if traffic_mode != 'fd' or confidence_level != '1':
                        continue
                        
                    thput_str = items[-6]
                    if thput_str == 'None':
                        continue
                        
                    thput = float(thput_str)
                    if thput < 0.1:
                        continue
                        
                    if cell_set_type == 'LTE+MW':
                        mw_thput_list.append(thput)
                    elif cell_set_type == 'LTE+Sub6':
                        sub6_thput_list.append(thput)
                    elif cell_set_type == 'LTE':
                        lte_thput_list.append(thput)
                        
        '''
        mw_rsrp_list = []
        sub6_rsrp_list = []
        lte_rsrp_list = []
        
        for k, v in rsrp_grid.items():
            mw_rsrp = v[2]
            sub6_rsrp = v[3]
            lte_rsrp = v[4]
            
            if mw_rsrp > -140:
                mw_rsrp_list.append(mw_rsrp)
                
            if sub6_rsrp > -140:
                sub6_rsrp_list.append(sub6_rsrp)
                
            if lte_rsrp > -140:
                lte_rsrp_list.append(lte_rsrp)
                
        '''
                
        mw_thput_0 = np.percentile(mw_thput_list, 10) if len(mw_thput_list) else 0
        mw_thput_25 = np.percentile(mw_thput_list, 25) if len(mw_thput_list) else 0
        mw_thput_50 = np.percentile(mw_thput_list, 50) if len(mw_thput_list) else 0
        mw_thput_75 = np.percentile(mw_thput_list, 75) if len(mw_thput_list) else 0
        mw_thput_100 = np.percentile(mw_thput_list, 90) if len(mw_thput_list) else 0

        sub6_thput_0 = np.percentile(sub6_thput_list, 10) if len(sub6_thput_list) else 0
        sub6_thput_25 = np.percentile(sub6_thput_list, 25) if len(sub6_thput_list) else 0
        sub6_thput_50 = np.percentile(sub6_thput_list, 50) if len(sub6_thput_list) else 0
        sub6_thput_75 = np.percentile(sub6_thput_list, 75) if len(sub6_thput_list) else 0
        sub6_thput_100 = np.percentile(sub6_thput_list, 90) if len(sub6_thput_list) else 0

        lte_thput_0 = np.percentile(lte_thput_list, 10) if len(lte_thput_list) else 0
        lte_thput_25 = np.percentile(lte_thput_list, 25) if len(lte_thput_list) else 0
        lte_thput_50 = np.percentile(lte_thput_list, 50) if len(lte_thput_list) else 0
        lte_thput_75 = np.percentile(lte_thput_list, 75) if len(lte_thput_list) else 0
        lte_thput_100 = np.percentile(lte_thput_list, 90) if len(lte_thput_list) else 0

        fout = open(path + "/" + operator + "_thput_boxplot_noaggr.csv", 'w')
        fout.write("rat,thput_10,thput_25,thput_50,thput_75,thput_90\n")
        fout.write('mw' + ',' + str(mw_thput_0) + ',' + str(mw_thput_25) + ',' + str(mw_thput_50) + ',' + str(mw_thput_75) + ',' + str(mw_thput_100) + '\n')
        fout.write('sub6' + ',' + str(sub6_thput_0) + ',' + str(sub6_thput_25) + ',' + str(sub6_thput_50) + ',' + str(sub6_thput_75) + ',' + str(sub6_thput_100) + '\n')
        fout.write('lte' + ',' + str(lte_thput_0) + ',' + str(lte_thput_25) + ',' + str(lte_thput_50) + ',' + str(lte_thput_75) + ',' + str(lte_thput_100) + '\n')
        fout.close()

        outlier_mw = []

        fout = open(path + "/" + operator + "_mw_thput_cdf_noaggr.csv", 'w')
        mw_thput_list.sort()
        length = len(mw_thput_list)
        cdf_value = 0
        step = 0
        for i in range(length):
            cdf_value += 1/length
            if cdf_value > step:
                step += 0.01
                fout.write(str(mw_thput_list[i]) + ','+ str(cdf_value) + '\n')
                if mw_thput_list[i] < mw_thput_0 or mw_thput_list[i] > mw_thput_100:
                    outlier_mw.append(mw_thput_list[i])
        fout.close()

        outlier_sub6 = []

        fout = open(path + "/" + operator + "_sub6_thput_cdf_noaggr.csv", 'w')
        sub6_thput_list.sort()
        length = len(sub6_thput_list)
        cdf_value = 0
        step = 0
        for i in range(length):
            cdf_value += 1/length
            if cdf_value > step:
                step += 0.01
                fout.write(str(sub6_thput_list[i]) + ','+ str(cdf_value) + '\n')
                if sub6_thput_list[i] < sub6_thput_0 or sub6_thput_list[i] > sub6_thput_100:
                    outlier_sub6.append(sub6_thput_list[i])
        fout.close()

        outlier_lte = []

        fout = open(path + "/" + operator + "_lte_thput_cdf_noaggr.csv", 'w')
        lte_thput_list.sort()
        length = len(lte_thput_list)
        cdf_value = 0
        step = 0
        for i in range(length):
            cdf_value += 1/length
            if cdf_value > step:
                step += 0.01
                fout.write(str(lte_thput_list[i]) + ','+ str(cdf_value) + '\n')
                if lte_thput_list[i] < lte_thput_0 or lte_thput_list[i] > lte_thput_100:
                    outlier_lte.append(lte_thput_list[i])
        fout.close()

        fout = open(path + "/" + operator + "_outlier_thput_noaggr.csv", 'w')
        for item in outlier_mw:
            fout.write('1,' + str(item) + '\n')
        for item in outlier_sub6:
            fout.write('2,' + str(item) + '\n')
        for item in outlier_lte:
            fout.write('3,' + str(item) + '\n')
        fout.close()


A-R1
A-R2
A-R3
A-R4
A-R5
V-R6
V-R7
V-R8
V-R9
V-R10
V-R11
T-R12
T-R13
